<a href="https://colab.research.google.com/github/SouravDasz/sentiment-analysis-tool/blob/main/syntiement_analysis_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.76GB/s]


In [5]:
import zipfile
zip=zipfile.ZipFile('sentiment140.zip')
zip.extractall("/content")
zip.close()



In [6]:
import numpy as np
import pandas as pd

In [7]:
import pandas as pd

df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
df.head()


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df=df.iloc[:,[0,5]]

In [9]:
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [10]:
df.columns=["label","text"]

In [11]:
df.head()

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [12]:
df["label"].value_counts()

,count
label,
0,800000
4,800000


In [13]:
import re

In [14]:
def process(x):
  x=re.sub(r'[^a-zA-Z]',' ',x)
  return x.lower()

In [15]:
x=df["text"].apply(process)

In [16]:
df

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [17]:
y=df["label"].map({0:0, 4:1})

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [20]:
from keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [21]:
tokenizer=Tokenizer(oov_token="oov")

In [22]:
tokenizer.fit_on_texts(x_train)

In [23]:
x_train=tokenizer.texts_to_sequences(x_train)
x_train=pad_sequences(x_train,padding="post")
x_test=tokenizer.texts_to_sequences(x_test)
x_test=pad_sequences(x_test,padding="post")

In [24]:
len(tokenizer.word_counts)

508283

In [25]:
max_len=len(tokenizer.word_index)

In [26]:
x_train

array([[40038,     2,   366, ...,     0,     0,     0],
       [  356,    83, 12762, ...,     0,     0,     0],
       [ 3178,     4, 13747, ...,     0,     0,     0],
       ...,
       [15546,     2,   376, ...,     0,     0,     0],
       [  234,     3,  9825, ...,     0,     0,     0],
       [   86,    67,    66, ...,     0,     0,     0]], dtype=int32)

In [27]:
from keras import Sequential
from keras.layers import LSTM , SimpleRNN,Dense,Embedding

In [28]:
model=Sequential()
model.add(Embedding(input_dim= max_len+1,output_dim= 50))
model.add(LSTM(128))
model.add(Dense(1,activation="sigmoid"))

In [29]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),batch_size=500)

Epoch 1/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 70s 27ms/step - accuracy: 0.5247 - loss: 0.6861 - val_accuracy: 0.5005 - val_loss: 0.6933
Epoch 2/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 68s 27ms/step - accuracy: 0.5842 - loss: 0.6566 - val_accuracy: 0.8003 - val_loss: 0.4354
Epoch 3/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 69s 27ms/step - accuracy: 0.8154 - loss: 0.4077 - val_accuracy: 0.8159 - val_loss: 0.4030
Epoch 4/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 69s 27ms/step - accuracy: 0.8660 - loss: 0.3128 - val_accuracy: 0.8068 - val_loss: 0.4268
Epoch 5/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 69s 27ms/step - accuracy: 0.8995 - loss: 0.2418 - val_accuracy: 0.8082 - val_loss: 0.4500
Epoch 6/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 69s 27ms/step - accuracy: 0.9206 - loss: 0.1931 - val_accuracy: 0.8015 - val_loss: 0.5401
Epoch 7/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 69s 27ms/step - accuracy: 0.9339 - loss: 0.1604 - val_accuracy: 0.7972 - val_loss: 0.6007
Epoch 8/10
2560/2560 ━━━━━━━━━━━━━━━━━━━━ 69s 27ms/step - accuracy: 0.9442 -

In [42]:
text="i hate you"

text=pd.Series(text)

text=tokenizer.texts_to_sequences(text)
sequence_maxlen = x_train.shape[1]
text=pad_sequences(text,maxlen=sequence_maxlen, padding="post")
text

array([[  2, 152,   9,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0]], dtype=int32)

In [51]:
import tensorflow as tf

# Ensure the model's call method is wrapped in tf.function with explicit input_signature
# This forces TensorFlow to re-trace the graph with a known input shape.
# (None, sequence_maxlen) means batch size can vary, sequence length is fixed.
@tf.function(input_signature=[tf.TensorSpec(shape=[None, sequence_maxlen], dtype=tf.int32)])
def call_model_for_prediction(inputs):
    return model(inputs, training=False)

# Convert the input text (numpy array) to a TensorFlow tensor
input_tensor = tf.convert_to_tensor(text, dtype=tf.int32)

# Use the tf.function wrapped call for prediction
predictions = call_model_for_prediction(input_tensor)

predictions

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.07592159]], dtype=float32)>

In [55]:
# Assuming 'new_predictions' holds the model's output and 'new_text' is the input text

if new_predictions.numpy()[0][0] < 0.5:
    sentiment_label = "Negative"
else:
    sentiment_label = "Positive"

print(f"For the text '{new_text}', the model predicted a score of {new_predictions.numpy()[0][0]:.4f}.\nThis corresponds to a: {sentiment_label} sentiment.")

# You can also use this system for any other prediction:
# For example, using the prediction for 'i hate you'
# predictions = call_model_for_prediction(tf.convert_to_tensor(text, dtype=tf.int32))
# if predictions.numpy()[0][0] < 0.5:
#     sentiment_label_hate = "Negative"
# else:
#     sentiment_label_hate = "Positive"
# print(f"\nFor 'i hate you', the score was {predictions.numpy()[0][0]:.4f}, which is: {sentiment_label_hate} sentiment.")

For the text 'terrible', the model predicted a score of 0.0032.
This corresponds to a: Negative sentiment.


In [60]:
new_text_awful = "greate job sir "

# Process the new text
new_text_series_awful = pd.Series(new_text_awful)
new_text_sequences_awful = tokenizer.texts_to_sequences(new_text_series_awful)

# Pad the sequence to the same maxlen as the training data
new_text_padded_awful = pad_sequences(new_text_sequences_awful, maxlen=sequence_maxlen, padding="post")

print(new_text_padded_awful)

[[50109   335  1618     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]]


In [61]:
import tensorflow as tf

# Convert the padded numpy array to a TensorFlow tensor
new_input_tensor_awful = tf.convert_to_tensor(new_text_padded_awful, dtype=tf.int32)

# Use the tf.function wrapped call for prediction
new_predictions_awful = call_model_for_prediction(new_input_tensor_awful)

print(new_predictions_awful)

# Interpret the prediction
if new_predictions_awful.numpy()[0][0] < 0.5:
    sentiment_awful = "Negative"
else:
    sentiment_awful = "Positive"

print(f"The sentiment for '{new_text_awful}' is: {sentiment_awful} (score: {new_predictions_awful.numpy()[0][0]:.4f})")

tf.Tensor([[0.99490684]], shape=(1, 1), dtype=float32)
The sentiment for 'greate job sir ' is: Positive (score: 0.9949)


In [62]:
new_text = "terrible fuck you"

# Process the new text similar to how 'i hate you' was processed
new_text_series = pd.Series(new_text)
new_text_sequences = tokenizer.texts_to_sequences(new_text_series)

# Pad the sequence to the same maxlen as the training data
new_text_padded = pad_sequences(new_text_sequences, maxlen=sequence_maxlen, padding="post")

print(new_text_padded)

[[1033  506    9    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


In [63]:
# Convert the padded numpy array to a TensorFlow tensor
new_input_tensor = tf.convert_to_tensor(new_text_padded, dtype=tf.int32)

# Use the tf.function wrapped call for prediction
new_predictions = call_model_for_prediction(new_input_tensor)

print(new_predictions)

# Interpret the prediction
if new_predictions.numpy()[0][0] < 0.5:
    sentiment = "Negative"
else:
    sentiment = "Positive"

print(f"The sentiment for '{new_text}' is: {sentiment} (score: {new_predictions.numpy()[0][0]:.4f})")

tf.Tensor([[0.00457629]], shape=(1, 1), dtype=float32)
The sentiment for 'terrible fuck you' is: Negative (score: 0.0046)


In [64]:
# Save the trained model
model.save('sentiment_model.keras')

print("Model saved successfully to 'sentiment_model.keras'")

Model saved successfully to 'sentiment_model.keras'


In [65]:
from tensorflow import keras

# Load the saved model
loaded_model = keras.models.load_model('sentiment_model.keras')

print("Model loaded successfully!")
loaded_model.summary()

Model loaded successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 53, 50)           │    25,414,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 128)              │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,517,933 (291.89 MB)

 Trainable params: 25,505,977 (97.30 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 51,011,956 (194.60 MB)

In [66]:
# Now you can use the loaded_model for predictions
# Let's test it with a new text, for example 'i am very happy today'

new_test_text = "i am very happy today"

# Process the new text similar to how the training data was processed
new_test_text_series = pd.Series(new_test_text)
new_test_text_sequences = tokenizer.texts_to_sequences(new_test_text_series)
new_test_text_padded = pad_sequences(new_test_text_sequences, maxlen=sequence_maxlen, padding="post")

# Convert to TensorFlow tensor
input_tensor_loaded = tf.convert_to_tensor(new_test_text_padded, dtype=tf.int32)

# Use the tf.function wrapped call for prediction (you can reuse the existing one)
predictions_loaded = call_model_for_prediction(input_tensor_loaded)

# Interpret the prediction
if predictions_loaded.numpy()[0][0] < 0.5:
    sentiment_loaded = "Negative"
else:
    sentiment_loaded = "Positive"

print(f"\nFor the text '{new_test_text}', the loaded model predicted a score of {predictions_loaded.numpy()[0][0]:.4f}.\nThis corresponds to a: {sentiment_loaded} sentiment.")


For the text 'i am very happy today', the loaded model predicted a score of 0.9994.
This corresponds to a: Positive sentiment.
